In [1]:
import gurobipy as gp
import math
from matplotlib import pyplot as plt
from matplotlib.ticker import FuncFormatter, PercentFormatter
import numpy as np
import pandas as pd
import re
import os
import shutil

In [2]:
# output generation for paper 2

In [3]:
# get input paths
test_set = "miplib_2017_5000_paper2"
instance_fldr = os.path.join("instances", test_set)
test_set_fldr = os.path.join("test_sets", test_set)
results_fldr = os.path.join("results", test_set)
out_fldr = os.path.join("outputs", test_set)

# set filters
seed_idxs = [0]  
max_indices = 4
degrees = [0, 2]  # todo update this as needed
term_list = [4, 64]
filter_cbc = False
max_base_std = 1e10
min_termination_time = 1
short, medium, long = 60, 600, 3600
remove_status_changes = False
win_threshold = .1

generators = ["None", "New", "Farkas", "All"]  # , "Disjunction", "Matrix", "Term", "Basis"]  #, "NoDisjunction", "NoMatrix", "NoTerm", "NoBasis"]

# set up some mappings
cat_map_new_lines = {
    "None": "Default",
    "Farkas": "Param Disj,\nParam Cuts",
    "Old": "Param Disj,\nCalc Cuts",
    "New": "Calc Disj,\nCalc Cuts"
}
cat_map = {
    "None": "Default",
    "Farkas": "Param Disj, Param Cuts",
    "Old": "Param Disj, Calc Cuts",
    "New": "Calc Disj, Calc Cuts"
}
perturbation_map = {
    "matrix": "Coefficient Matrix",
    "rhs": "Right Hand Side",
    "objective": "Objective"
}
label = {
    "postRootTime": "Time after Processing Root nodes",
    "rootDualBoundTimeSansVpc": "Root Processing Time (Minus VPC Generation)",
    "terminationTimeSansVpc": "Time (Minus VPC Generation)",
    "terminationTime": "Time",
    "nodes": "Nodes Processed",
    "iterations": "LP iterations",
}
unit = {
    "postRootTime": "(seconds)",
    "rootDualBoundTimeSansVpc": "(seconds)",
    "terminationTimeSansVpc": "(seconds)",
    "terminationTime": "(seconds)",
    "nodes": "(1000 nodes)",
    "iterations": "(1000 iterations)",
}
limits = {
    "postRootTime": 7200,
    "terminationTimeSansVpc": 7200,
    "terminationTime": 7200,
    "rootDualBoundTimeSansVpc": 5,
    "nodes": 10000,
    "iterations": 37500
}
bracket_bounds = {
    "short": (min_termination_time, short),
    "medium": (short, medium),
    "long": (medium, long)
}
param_map = {
    "degree": "Degree of Perturbation",
    "terms": "Number of Disjunctive Terms",
}

In [4]:
# matplotlib settings
plt.rc('text', usetex=True)  # use latex fonts
plt.rcParams['font.size'] = 18
plt.rcParams['figure.titlesize'] = 24
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16
plt.rcParams['legend.fontsize'] = 14

## Check run failures

In [5]:
# check if each folder in test_set_fldr has a corresponding .mps file in instance_fldr
# for instance in os.listdir(test_set_fldr):
#     if not os.path.isdir(os.path.join(test_set_fldr, instance)):
#         continue
#     if not os.path.exists(os.path.join(instance_fldr, f"{instance}.mps")):
#         # remove the folder if the instance is missing
#         # shutil.rmtree(os.path.join(test_set_fldr, instance))
#         print(f"Removed {instance} from test set")

In [6]:
# running list of strings contained by different error codes
# last two are catchalls
err = {
    "walltime": [],
    "bad_alloc": [],
    "out of memory": [],
    "vmem": [],
    "takeoffcuts": [],
    "solver is dual infeasible": [],
    "solver must be optimal": [],
    "segmentation fault": [],
    "no vpcs were made from a new disjunction": [],
    "must have primalbound >= root lp objective": [],
    "objective at parent nodes": [],
    "failed to optimize mip": [],
    "disjunction does not represent a full binary tree": [],
    "solver not proven optimal for nodes": [],
    "unable to open": [],
    "license": [],
    "dot product with obj differs from solver": [],
    "gurobi: error during callback: addCut": [],
    "cglvpc::setupconstraints: objective at disjunctive term": [],
    "unable to read file": [],
    "stats.id == stats_vec": [],
    "size of our disjunction is not what we expected it to be": [],
    "dimension must stay fixed": [],
    "vpcgenerator must be": [],
    "objective values must match": [],
    "objective at disjunctive term": [],
}

# read in cbc acceptable instances from cbc.txt
with open("cbc.txt", "r") as f:
    cbc_instances = f.read().split("\n")

# runs that errored out with new error code
other = []

# runs that had no errors
empty = []

# runs that only had warnings
warn_strs = ["warning", "prlp is primal infeasible", "farkas", "x:", "x[", "b:",
             "b[", "v:", "v[", "cut:", "A_i . x", "dot product with obj differs from solver"]
warning = []

# series that didn't run
no_go = []

# track sizes of instances
rows, cols, density = {}, {}, {}

# map the names
names = {}

# counts
count_series = 0
count_instances = 0
number_instances = {}

# iterate over all expected runs
for instance in os.listdir(test_set_fldr):
    if not os.path.isdir(os.path.join(test_set_fldr, instance)):
        continue
    # only look at cbc instances if we ran with cbc
    if instance not in cbc_instances and "gurobi" not in test_set and filter_cbc:
        continue
        
    # get the number of rows and columns in the instance
    mdl = gp.read(os.path.join(instance_fldr, f"{instance}.mps"))
    rows[instance] = mdl.NumConstrs
    cols[instance] = mdl.NumVars
    density[instance] = mdl.NumNZs / (mdl.NumConstrs * mdl.NumVars)
        
    for perturbation in os.listdir(os.path.join(test_set_fldr, instance)):
        if not os.path.isdir(os.path.join(test_set_fldr, instance, perturbation)):
            continue
        # only look at perturbations that were run
        p, d = perturbation.split("_")
        if int(d) not in degrees:
            continue
        for terms in term_list:
            for generator in generators:
                for seed_idx in seed_idxs:

                    # set variables for this iterations
                    count_series += 1
                    stem = f"{instance}_{perturbation}_{terms}_{generator}_{seed_idx}"
                    file_pth = os.path.join(results_fldr, f"{stem}.err")
                    series_fldr = os.path.join(test_set_fldr, instance, perturbation)
                    current_count = len([f for f in os.listdir(series_fldr) if f.endswith(".mps")])
                    count_instances += current_count
                    names[stem] = instance
                    number_instances[stem] = {
                        "expected": current_count,
                        "recorded": 0,
                        "generator": generator,
                        "error": "N/A"
                    }
    
                    # check if the series wasn't run
                    if not os.path.exists(file_pth):
                        number_instances[stem]["error"] = "no go"
                        no_go.append(stem)
                    
                    # check if the series ran with no errors or warnings
                    elif os.path.getsize(file_pth) == 0:
                        number_instances[stem]["error"] = "empty"
                        empty.append(stem)
                    
                    # track which error codes were thrown
                    else:
                        # read the file
                        with open(file_pth, "r") as f:
                            text = f.read().lower()
                        
                        # assign the error file to the appropriate list
                        found_code = False
                        for code in err:
                            if code in text:
                                if code == "dot product with obj differs from solver":
                                    pattern = r"obj viol from solver: (-?\d+\.\d+)\. calculated: (-?\d+\.\d+)"
                                    s, c = re.findall(pattern, text)[-1]
                                    # if we didn't terminate, this isn't an error, so keep going
                                    if abs(float(s) - float(c)) < 1e-3:
                                        continue
                                err[code].append(stem)
                                found_code = True
                                number_instances[stem]["error"] = code
                                break
                        if not found_code:
                            if all(not line or any(w in line for w in warn_strs) for line in text.splitlines()):
                                warning.append(stem)
                                number_instances[stem]["error"] = "warning"
                            else:
                                other.append(stem)
                                number_instances[stem]["error"] = "other"

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-21
Read MPS format model from file instances/miplib_2017_5000_v2/bienst2.mps
Reading time = 0.00 seconds
bienst2: 576 rows, 505 columns, 2184 nonzeros
Read MPS format model from file instances/miplib_2017_5000_v2/f2gap801600.mps
Reading time = 0.00 seconds
f2gap801600: 80 rows, 1600 columns, 3200 nonzeros
Read MPS format model from file instances/miplib_2017_5000_v2/neos-3610173-itata.mps
Reading time = 0.00 seconds
neos-3610173-itata: 747 rows, 844 columns, 2130 nonzeros
Read MPS format model from file instances/miplib_2017_5000_v2/10teams.mps
Reading time = 0.01 seconds
10teams: 230 rows, 2025 columns, 12150 nonzeros
Read MPS format model from file instances/miplib_2017_5000_v2/gmu-35-40.mps
Reading time = 0.00 seconds
gmu-35-40: 424 rows, 1205 columns, 4843 nonzeros
Read MPS format model from file instances/miplib_2017_5000_v2/neos-3610051-istra.mps
Reading time = 0.00 seconds
neos-3610051-istra:

In [7]:
# check which series didn't run
print(no_go)

[]


In [8]:
# get the proportion of series that at least got started
1 - (len(no_go) / count_series)

1.0

In [9]:
# out of time - got hung up in code somewhere - ok
print(err["walltime"])
len(err["walltime"]) / count_series

['cod105_rhs_0_64_New_0', 'cod105_rhs_0_64_All_0', 'cod105_rhs_2_64_New_0', 'cod105_rhs_2_64_Farkas_0', 'cod105_objective_2_64_New_0']


0.00048003072196620584

In [10]:
# out of memory - memory is maxed already - this is what it is
# todo: figure out where we ran short on memory so we can explain why we dropped them
print(err["bad_alloc"] + err["out of memory"] + err["vmem"])
len(err["bad_alloc"] + err["out of memory"] + err["vmem"]) / count_series

['10teams_rhs_2_64_New_0', '10teams_objective_2_64_New_0', 'aflow40b_matrix_2_4_New_0', 'aflow40b_matrix_2_64_New_0', 'piperout-d27_objective_0_64_New_0', 'piperout-d27_objective_0_64_Farkas_0', 'piperout-d27_objective_0_64_All_0', 'piperout-d27_objective_2_64_New_0', 'piperout-d27_objective_2_64_Farkas_0', 'piperout-d27_objective_2_64_All_0', 'piperout-d20_objective_0_64_New_0', 'piperout-d20_objective_0_64_Farkas_0', 'piperout-d20_objective_0_64_All_0', 'piperout-d20_objective_2_64_New_0', 'piperout-d20_objective_2_64_Farkas_0', 'piperout-d20_objective_2_64_All_0', 'nexp-150-20-1-5_matrix_2_64_New_0', 'qnet1_matrix_0_64_New_0', 'neos-2328163-agri_objective_0_64_New_0', 'neos-2328163-agri_objective_2_64_New_0', 'mod010_matrix_0_64_New_0', 'mod010_objective_0_64_New_0', 'mod010_objective_0_64_Farkas_0', 'mod010_rhs_2_4_New_0', 'mod010_rhs_2_64_New_0', 'mod010_rhs_2_64_Farkas_0', 'mod010_rhs_2_64_All_0', 'mod010_objective_2_64_New_0', 'app2-2_rhs_0_64_Farkas_0', 'app2-2_rhs_0_64_All_0',

0.020737327188940093

In [11]:
# rerun this if want to give more memory to some instances
# bad_alloc_names = set(n.split("_")[0] for n in err["bad_alloc"])
# mem = pd.read_csv("more_memory.csv", index_col=0)
# mem["reason"] = "hard solve" 
# 
# for n in bad_alloc_names:
#     if f"{n}.mps" not in mem.index:
#         new_row = pd.DataFrame([{'file_name': f"{n}.mps", 'memory': 16.0, 'reason': 'big disjunction'}]).set_index('file_name')
#         mem = pd.concat([mem, new_row])
#     else:
#         mem.loc[f'{n}.mps', 'memory'] = 16.0
# 
# mem.to_csv("more_memory.csv")

In [12]:
# this is an issue with John's bookkeeping - not much we can do here
print(err["takeoffcuts"])
len(err["takeoffcuts"]) / count_series

[]


0.0

In [13]:
print(err["solver is dual infeasible"])
len(err["solver is dual infeasible"]) / count_series

[]


0.0

In [14]:
# these are usually issues with CLP finding optimality - not much we can do here
print(err["solver must be optimal"])
len(err["solver must be optimal"]) / count_series

[]


0.0

In [15]:
print(err["segmentation fault"])
len(err["segmentation fault"]) / count_series

['neos-3665875-lesum_rhs_0_64_All_0', 'neos-3665875-lesum_matrix_0_64_Farkas_0', 'neos-3665875-lesum_objective_0_64_New_0', 'neos-3665875-lesum_objective_0_64_Farkas_0', 'neos-3665875-lesum_rhs_2_64_New_0', 'neos-3665875-lesum_objective_2_64_New_0', 'neos-3665875-lesum_matrix_2_64_All_0']


0.0006720430107526882

In [16]:
# seg_err = {
#     "Bad image at line": [],
# }
# 
# seg_other = []
# 
# for stem in err["segmentation fault"]:
#     file_pth = os.path.join(results_fldr, f"{stem}.out")
# 
#     with open(file_pth, "r") as f:
#         text = f.read()
#     
#     # assign the error file to the appropriate list
#     found_code = False
#     for code in seg_err:
#         if code in text:
#             seg_err[code].append(stem)
#             found_code = True
#             break
#     if not found_code:
#         seg_other.append(stem)

In [17]:
# print(seg_err["Bad image at line"])
# len(seg_err["Bad image at line"]) / len(err["segmentation fault"]) if err["segmentation fault"] else 0

In [18]:
# print(seg_other)
# len(seg_other)/len(err["segmentation fault"]) if err["segmentation fault"] else 0

In [19]:
# # get breakdown of why vpc generation failed - mostly from lack of provisioning
# for code, exps in seg_err.items():
#     print(f"{code}: {len(exps) / len(err['segmentation fault']) if err['segmentation fault'] else 0}")
# 
# print(f"other: {len(seg_other) / len(err['segmentation fault']) if err['segmentation fault'] else 0}")

In [20]:
# todo: check aleks' removals and drop those below for similar reasons
# todo: check size of disjunctions and decide what to do with those that are too big
# these should all be from the problem being too big and hitting the time limit or integer solutions
print(err["no vpcs were made from a new disjunction"])
missing_4_term = [n for n in err["no vpcs were made from a new disjunction"] if "_4_" in n]
missing_64_term = [n for n in err["no vpcs were made from a new disjunction"] if "_64_" in n]
print(f'4 term: {len(missing_4_term) / count_series}')
print(f'64 term: {len(missing_64_term) / count_series}')

['bienst2_rhs_0_64_New_0', 'bienst2_rhs_0_64_Farkas_0', 'bienst2_rhs_0_64_All_0', 'bienst2_matrix_0_64_New_0', 'bienst2_matrix_0_64_All_0', 'bienst2_objective_0_64_New_0', 'bienst2_objective_0_64_Farkas_0', 'bienst2_rhs_2_64_New_0', 'bienst2_rhs_2_64_Farkas_0', 'bienst2_rhs_2_64_All_0', 'bienst2_objective_2_64_Farkas_0', 'bienst2_objective_2_64_All_0', 'bienst2_matrix_2_64_Farkas_0', 'bienst2_matrix_2_64_All_0', '10teams_matrix_2_4_Farkas_0', 'neos-555343_rhs_0_4_New_0', 'neos-555343_rhs_0_4_Farkas_0', 'neos-555343_rhs_0_4_All_0', 'neos-555343_rhs_0_64_New_0', 'neos-555343_rhs_0_64_Farkas_0', 'neos-555343_rhs_0_64_All_0', 'neos-555343_matrix_0_4_New_0', 'neos-555343_matrix_0_4_Farkas_0', 'neos-555343_matrix_0_4_All_0', 'neos-555343_matrix_0_64_New_0', 'neos-555343_matrix_0_64_Farkas_0', 'neos-555343_matrix_0_64_All_0', 'neos-555343_objective_0_4_New_0', 'neos-555343_objective_0_4_Farkas_0', 'neos-555343_objective_0_4_All_0', 'neos-555343_objective_0_64_New_0', 'neos-555343_objective_0_

In [21]:
# vpc_err = {
#     "CglVPC: Finishing with exit reason: PRLP_TIME_LIMIT": [],
#     "CglVPC: Finishing with exit reason: TIME_LIMIT": [],
#     "CglVPC: Finishing with exit reason: NO_CUTS_LIKELY": [],
#     "CglVPC: Finishing with exit reason: PRLP_INFEASIBLE": [],
#     "CglVPC: Finishing with exit reason: SUCCESS": [],
#     "CglVPC: Finishing with exit reason: OPTIMAL_SOLUTION_FOUND": [],
#     "CglVPC: Finishing with exit reason: FAIL_LIMIT": [],
#     "CglVPC: Finishing with exit reason: NO_DISJUNCTION": [],
# }
# 
# vpc_other = []
# 
# for stem in err["no vpcs were made from a new disjunction"]:
#     file_pth = os.path.join(results_fldr, f"{stem}.out")
# 
#     with open(file_pth, "r") as f:
#         text = f.read()
#     
#     # assign the error file to the appropriate list
#     found_code = False
#     for code in vpc_err:
#         if code in text:
#             vpc_err[code].append(stem)
#             found_code = True
#             break
#     if not found_code:
#         vpc_other.append(stem)

In [22]:
# print(vpc_err["CglVPC: Finishing with exit reason: PRLP_TIME_LIMIT"])
# if err["no vpcs were made from a new disjunction"]:
#     len(vpc_err["CglVPC: Finishing with exit reason: PRLP_TIME_LIMIT"]) / len(err["no vpcs were made from a new disjunction"])

In [23]:
# print(vpc_err["CglVPC: Finishing with exit reason: TIME_LIMIT"])
# if err["no vpcs were made from a new disjunction"]:
#     len(vpc_err["CglVPC: Finishing with exit reason: TIME_LIMIT"]) / len(err["no vpcs were made from a new disjunction"])

In [24]:
# print(vpc_err["CglVPC: Finishing with exit reason: NO_CUTS_LIKELY"])
# if err["no vpcs were made from a new disjunction"]:
#     len(vpc_err["CglVPC: Finishing with exit reason: NO_CUTS_LIKELY"]) / len(err["no vpcs were made from a new disjunction"])

In [25]:
# print(vpc_err["CglVPC: Finishing with exit reason: PRLP_INFEASIBLE"])
# if err["no vpcs were made from a new disjunction"]:
#     len(vpc_err["CglVPC: Finishing with exit reason: PRLP_INFEASIBLE"]) / len(err["no vpcs were made from a new disjunction"])

In [26]:
# print(vpc_err["CglVPC: Finishing with exit reason: SUCCESS"])
# if err["no vpcs were made from a new disjunction"]:
#     len(vpc_err["CglVPC: Finishing with exit reason: SUCCESS"]) / len(err["no vpcs were made from a new disjunction"])

In [27]:
# print(vpc_err["CglVPC: Finishing with exit reason: OPTIMAL_SOLUTION_FOUND"])
# if err["no vpcs were made from a new disjunction"]:
#     len(vpc_err["CglVPC: Finishing with exit reason: OPTIMAL_SOLUTION_FOUND"]) / len(err["no vpcs were made from a new disjunction"])

In [28]:
# print(vpc_err["CglVPC: Finishing with exit reason: FAIL_LIMIT"])
# if err["no vpcs were made from a new disjunction"]:
#     len(vpc_err["CglVPC: Finishing with exit reason: FAIL_LIMIT"]) / len(err["no vpcs were made from a new disjunction"])

In [29]:
# print(vpc_err["CglVPC: Finishing with exit reason: NO_DISJUNCTION"])
# if err["no vpcs were made from a new disjunction"]:
#     len(vpc_err["CglVPC: Finishing with exit reason: NO_DISJUNCTION"]) / len(err["no vpcs were made from a new disjunction"])

In [30]:
# vpc_other

In [31]:
# # get breakdown of why vpc generation failed - mostly from lack of provisioning/problem being too large
# if err["no vpcs were made from a new disjunction"]:
#     for code, exps in vpc_err.items():
#         print(f"{code}: {len(exps) / len(err['no vpcs were made from a new disjunction'])}")
#     
#     print(f"other: {len(vpc_other) / len(err['no vpcs were made from a new disjunction'])}")

In [32]:
print(err["must have primalbound >= root lp objective"])
len(err["must have primalbound >= root lp objective"]) / count_series

['neos-3421095-cinca_matrix_0_4_None_0', 'neos-3421095-cinca_matrix_0_4_New_0', 'neos-3421095-cinca_matrix_0_4_Farkas_0', 'neos-3421095-cinca_matrix_0_4_All_0', 'neos-3421095-cinca_matrix_0_64_None_0', 'neos-3421095-cinca_matrix_0_64_New_0', 'neos-3421095-cinca_matrix_0_64_Farkas_0', 'neos-3421095-cinca_matrix_0_64_All_0']


0.0007680491551459293

In [33]:
# LP relaxation objective is not going to match root nodes objective when warm starting 
print(err["objective at parent nodes"])
len(err["objective at parent nodes"]) / count_series

[]


0.0

In [34]:
# not enough tolerance added to bound (or we hit time limit) - element 2 from 5 and 4 from 4
print(err["failed to optimize mip"])
len(err["failed to optimize mip"]) / count_series

['f2gap40400_objective_0_4_New_0', 'markshare_5_0_matrix_2_4_None_0', 'markshare_5_0_matrix_2_64_None_0', 'neos-3373491-avoca_objective_0_64_None_0']


0.00038402457757296467

In [35]:
# todo: figure out why
print(err["disjunction does not represent a full binary tree"])
len(err["disjunction does not represent a full binary tree"]) / count_series

['nu25-pr12_matrix_2_64_New_0', 'p0201_objective_0_64_New_0']


0.00019201228878648233

In [36]:
# again issue with not getting through vpc generation in time
# todo: handle this gracefully
print(err["solver not proven optimal for nodes"])
len(err["solver not proven optimal for nodes"]) / count_series

[]


0.0

In [37]:
print(err["unable to open"])
len(err["unable to open"]) / count_series

[]


0.0

In [38]:
print(err["license"])
len(err["license"]) / count_series

[]


0.0

In [39]:
print(warning)
len(warning) / count_series

['bienst2_rhs_0_4_Farkas_0', 'bienst2_matrix_0_4_New_0', 'bienst2_matrix_0_4_Farkas_0', 'bienst2_objective_0_4_New_0', 'bienst2_rhs_2_4_New_0', 'bienst2_objective_2_4_New_0', 'bienst2_objective_2_64_New_0', 'bienst2_matrix_2_4_New_0', 'bienst2_matrix_2_4_All_0', 'f2gap801600_rhs_0_64_New_0', 'f2gap801600_objective_0_64_New_0', 'neos-3610173-itata_matrix_0_4_New_0', 'neos-3610173-itata_matrix_0_64_New_0', '10teams_rhs_0_4_New_0', '10teams_rhs_0_64_New_0', '10teams_rhs_0_64_Farkas_0', '10teams_matrix_0_64_New_0', '10teams_matrix_0_64_Farkas_0', '10teams_objective_0_4_New_0', '10teams_objective_0_64_All_0', '10teams_rhs_2_64_All_0', '10teams_objective_2_4_New_0', 'gmu-35-40_rhs_0_4_New_0', 'gmu-35-40_rhs_0_4_Farkas_0', 'gmu-35-40_rhs_0_4_All_0', 'gmu-35-40_matrix_0_4_New_0', 'gmu-35-40_matrix_0_4_Farkas_0', 'gmu-35-40_matrix_0_4_All_0', 'gmu-35-40_objective_0_4_New_0', 'gmu-35-40_objective_0_4_Farkas_0', 'gmu-35-40_objective_0_4_All_0', 'gmu-35-40_objective_0_64_New_0', 'gmu-35-40_objecti

0.04992319508448541

In [40]:
# errors unaccounted for
print(other)
len(other) / count_series

['eil33-2_objective_2_4_New_0', 'neos-4333596-skien_objective_2_64_New_0']


0.00019201228878648233

In [41]:
# proportion of series that were improperly provisioned
(len(err["bad_alloc"] + err["out of memory"] + err["walltime"] + err["vmem"])) / count_series

0.021217357910906297

In [42]:
# todo handle this
print(err["dot product with obj differs from solver"])
len(err["dot product with obj differs from solver"]) / count_series

['neos-480878_objective_2_4_New_0', 'neos-480878_objective_2_64_New_0', 'neos-860300_matrix_2_64_New_0']


0.0002880184331797235

In [43]:
# changed code to ignore this error
print(err["gurobi: error during callback: addCut"])
len(err["gurobi: error during callback: addCut"]) / count_series

[]


0.0

In [44]:
# largely not replicating - only issue I could find was aleks missing updated objective from CLP when resolving to check this
print(err["cglvpc::setupconstraints: objective at disjunctive term"])
len(err["cglvpc::setupconstraints: objective at disjunctive term"]) / count_series

[]


0.0

In [45]:
# not replicating - rerun
print(err["unable to read file"])
len(err["unable to read file"]) / count_series

[]


0.0

In [46]:
# not replicating - rerun
print(err["stats.id == stats_vec"])
len(err["stats.id == stats_vec"]) / count_series

[]


0.0

In [47]:
print(err["size of our disjunction is not what we expected it to be"])
len(err["size of our disjunction is not what we expected it to be"]) / count_series

[]


0.0

In [48]:
print(err["vpcgenerator must be"])
len(err["vpcgenerator must be"]) / count_series

[]


0.0

In [49]:
print(err["dimension must stay fixed"])
len(err["dimension must stay fixed"]) / count_series

[]


0.0

In [50]:
print(err["objective values must match"])
len(err["objective values must match"]) / count_series

['mas74_matrix_0_4_New_0', 'mas74_matrix_0_64_New_0', 'mas74_rhs_2_4_New_0', 'mas74_rhs_2_64_New_0', 'mas74_matrix_2_64_New_0', 'bppc8-09_matrix_0_64_New_0', 'rentacar_matrix_0_4_New_0', 'neos-3421095-cinca_objective_0_64_New_0', 'neos-3610040-iskar_matrix_0_64_New_0', 'neos-3627168-kasai_matrix_0_64_New_0', 'neos-5182409-nasivi_matrix_0_4_New_0', 'neos-5182409-nasivi_matrix_0_64_New_0', 'mas76_matrix_0_4_New_0', 'mas76_matrix_0_64_New_0', 'mas76_matrix_2_64_New_0', 'binkar10_1_rhs_0_64_New_0', 'neos-3592146-hawea_matrix_2_4_New_0', 'supportcase25_rhs_2_4_New_0', 'neos-3754480-nidda_objective_0_4_New_0', 'neos-3754480-nidda_objective_0_64_New_0', 'neos-3754480-nidda_objective_2_64_New_0', 'control30-3-2-3_matrix_0_4_New_0', 'neos-4333596-skien_matrix_0_64_New_0', 'neos-3072252-nete_matrix_0_64_New_0']


0.002304147465437788

In [51]:
print(err["objective at disjunctive term"])
len(err["objective at disjunctive term"]) / count_series

['neos-631517_matrix_0_4_New_0', 'neos-631517_matrix_0_64_New_0', 'gus-sch_matrix_2_4_New_0', 'gus-sch_matrix_2_64_New_0', 'dcmulti_matrix_2_4_New_0', 'fiber_matrix_2_4_New_0', 'fiber_matrix_2_64_New_0', 'roll3000_matrix_0_4_New_0', 'roll3000_matrix_0_64_New_0', 'supportcase25_rhs_0_4_New_0', 'mkc1_matrix_2_4_New_0', 'control30-3-2-3_matrix_0_64_New_0']


0.001152073732718894

In [52]:
# get breakdown of errors
for code, exps in err.items():
    print(f"{code}: {len(exps) / count_series}")

print(f"other: {len(other) / count_series}")

print(f"warning: {len(warning) / count_series}")

print(f"no errors/warnings: {len(empty) / count_series}")

print(f"no go: {len(no_go) / count_series}")

walltime: 0.00048003072196620584
bad_alloc: 0.018433179723502304
out of memory: 0.00038402457757296467
vmem: 0.0019201228878648233
takeoffcuts: 0.0
solver is dual infeasible: 0.0
solver must be optimal: 0.0
segmentation fault: 0.0006720430107526882
no vpcs were made from a new disjunction: 0.30184331797235026
must have primalbound >= root lp objective: 0.0007680491551459293
objective at parent nodes: 0.0
failed to optimize mip: 0.00038402457757296467
disjunction does not represent a full binary tree: 0.00019201228878648233
solver not proven optimal for nodes: 0.0
unable to open: 0.0
license: 0.0
dot product with obj differs from solver: 0.0002880184331797235
gurobi: error during callback: addCut: 0.0
cglvpc::setupconstraints: objective at disjunctive term: 0.0
unable to read file: 0.0
stats.id == stats_vec: 0.0
size of our disjunction is not what we expected it to be: 0.0
dimension must stay fixed: 0.0
vpcgenerator must be: 0.0
objective values must match: 0.002304147465437788
objectiv

## Read in data

In [53]:
# map generator names to the corresponding data frames
df_map = {g: pd.DataFrame() for g in generators} 
gap_map = {g: pd.DataFrame() for g in generators}
regex = re.compile(r'([a-zA-Z0-9-]+(?:_o)?)_([a-z]+)_([0-9-]+)_([0-9]+)_([a-zA-Z ]+)')
solution_pattern = r"_(\d+)\.pb"

# declaring types as needed
column_types = {
    "lpBound": float,
    "lpBoundPostVpc": float,
    "disjunctiveDualBound": float,
    "primalBound": float,
    "rootDualBound": float,
    "dualBound": float
}

skipped_instances = set()
primal_bounds = {}
same_solution = {}

# iterate over all files in the folder
for file_name in os.listdir(results_fldr):
    
    file_pth = os.path.join(results_fldr, file_name)
    
    # if the file is not a nonempty csv, skip it
    if not file_name.endswith(".csv") or os.path.getsize(file_pth) == 0:
        continue
    
    # get the experimental set up
    match = regex.search(file_name)
    instance_name = names.get(file_name[:-4])
    if not instance_name:
        skipped_instances.add(file_name[:-4].split("_")[0])
        os.remove(file_pth)
        continue
    # instance_name = match.group(1)
    perturbation = match.group(2)
    assert perturbation in ["matrix", "rhs", "bounds", "objective"], f"Unknown perturbation: {perturbation}"
    expo = int(match.group(3))
    assert expo in degrees, f"Unknown degree: {expo}"
    degree = 2**int(expo)
    terms = int(match.group(4))
    assert terms in term_list, f"Unknown number of terms: {terms}"
    generator = match.group(5)
    assert generator in generators, f"Unknown generator: {generator}"
    base_name = f"{instance_name}_0"
    
    # get the primal bounds for this experiment
    cur_instance_test_set_fldr = os.path.join(test_set_fldr, instance_name, f"{perturbation}_{expo}")
    for test_set_file in os.listdir(cur_instance_test_set_fldr):
        if test_set_file.endswith(".pb"):
            with open(os.path.join(cur_instance_test_set_fldr, test_set_file), "r") as f:
                primal_bounds[perturbation, expo, ".".join(test_set_file.split(".")[:-1])] = float(f.read())
                
    # see if solution changed
    for test_set_file in os.listdir(cur_instance_test_set_fldr):
        if test_set_file.endswith(".pb"):
            perturbation_name = ".".join(test_set_file.split(".")[:-1])
            same_solution[perturbation, expo, perturbation_name] = \
                primal_bounds[perturbation, expo, base_name] == primal_bounds[perturbation, expo, perturbation_name]
            
    # read the file
    df = pd.read_csv(file_pth, keep_default_na=False, dtype=column_types, index_col=0)
    
    for instance_idx in df.index:
        
        # fill in primal bounds if missing
        # df.loc[instance_idx, "primalBound"] = min(primal_bounds.get(stem_map.get(instance_idx), 1e100), df.loc[instance_idx, "primalBound"])
        df.loc[instance_idx, "primalBound"] = min(
            primal_bounds[perturbation, expo, f"{instance_name}_{instance_idx}"], df.loc[instance_idx, "primalBound"]
        )
        
        # same with root dual bound
        df.loc[instance_idx, "rootDualBound"] = df.loc[instance_idx, "rootDualBound"] if df.loc[instance_idx, "rootDualBound"] < 1e100 else df.loc[instance_idx, "lpBoundPostVpc"] 
    
    # get rid of the index so the rest of the notebook works
    df.reset_index(inplace=True)
    
    # add some identifying columns
    df["instance"] = instance_name
    df["perturbation"] = perturbation
    df["degree"] = degree
    df["terms"] = terms
    df["rows"] = rows[instance_name]
    df["cols"] = cols[instance_name]
    df["density"] = density[instance_name]
    
    # append to the appropriate data frame
    df_map[generator] = pd.concat([df_map[generator], df])
    
    # track recorded vs expected experiments
    number_instances[file_name[:-4]]["recorded"] = len(df)

In [54]:
# convert number_instances to dataframe
frame = pd.DataFrame(number_instances).T
frame.head()

,expected,recorded,generator,error
bienst2_rhs_0_4_None_0,4,4,None,empty
bienst2_rhs_0_4_New_0,4,4,New,empty
bienst2_rhs_0_4_Farkas_0,4,4,Farkas,warning
bienst2_rhs_0_4_All_0,4,4,All,empty
bienst2_rhs_0_64_None_0,4,4,None,empty


In [55]:
# redo the runs that have incomplete data that we're not sure should be that way
redos = frame.loc[(frame["expected"] > frame["recorded"]) & (frame["error"] != "no vpcs were made from a new disjunction")].index.tolist()
redos = pd.DataFrame({"experiment": redos})
redos.to_csv("redos.csv", index=False)

In [56]:
if "miplib" in test_set or "quick" in test_set:
    # group frame by generator and sum remaining columns
    gb = frame.groupby(["generator", "error"]).sum().reset_index()
    gb["missing"] = gb["expected"] - gb["recorded"]
    total = gb.groupby("generator")[["expected", "missing"]].sum().reset_index()
    gb = pd.merge(gb, total, on="generator", suffixes=("", " total"))
    gb["ratio missing (by generator)"] = gb["missing"] / gb["missing total"]
    gb["ratio missing (by generator)"] = gb["ratio missing (by generator)"].apply(lambda x: round(x, 4))
    gb = gb.loc[:, ~gb.columns.str.contains("total")]  # get rid of the total columns
    gb.set_index(["generator", "error"], inplace=True)
    gb.to_csv(os.path.join(out_fldr, "missing_table.csv"), index=False, mode="w")
else:
    gb = None
gb

expected recorded  \
generator error                                                                 
All       bad_alloc                                              129       35   
          empty                                                 5440     5151   
          must have primalbound >= root lp objective               8        6   
          no vpcs were made from a new disjunction              4042        0   
          out of memory                                            4        0   
          segmentation fault                                       8        0   
          vmem                                                    12        4   
          walltime                                                 4        0   
          warning                                                527      515   
Farkas    bad_alloc                                              133       34   
          empty                                                 5434     5434   
          must have primalbound >= root lp objective               8        6   
          no vpcs were made from a new disjunction              4054        0   
          out of memory                                            8        0   
          segmentation fault                                       8        0   
          vmem                                                    32        5   
          walltime                                                 4        0   
          warning                                                493      493   
New       bad_alloc                                              497      227   
          disjunction does not represent a full binary tree        8        5   
          dot product with obj differs from solver                12        9   
          empty                                                 4403     4291   
          failed to optimize mip                                   4        2   
          must have primalbound >= root lp objective               8        4   
          no vpcs were made from a new disjunction              4030        0   
          objective at disjunctive term                           48       30   
          objective values must match                             94       50   
          other                                                    8        5   
          out of memory                                            4        0   
          segmentation fault                                      12        0   
          vmem                                                    24        2   
          walltime                                                12        0   
          warning                                               1010      855   
None      empty                                                10142    10142   
          failed to optimize mip                                  12        7   
          must have primalbound >= root lp objective               8        6   
          vmem                                                    12        2   

                                                            missing  \
generator error                                                       
All       bad_alloc                                              94   
          empty                                                 289   
          must have primalbound >= root lp objective              2   
          no vpcs were made from a new disjunction             4042   
          out of memory                                           4   
          segmentation fault                                      8   
          vmem                                                    8   
          walltime                                                4   
          warning                                                12   
Farkas    bad_alloc                                              99   
          empty                                 

In [57]:
for gen in generators:
    masks = {
        0: -1e20 > df_map[gen]["lpBound"],
        1: df_map[gen]["lpBound"] - 1e-3 > df_map[gen]["lpBoundPostVpc"],
        2: (df_map[gen]["lpBoundPostVpc"] - 1e-3 > df_map[gen]["disjunctiveDualBound"]) & ((gen == "None") | (gen == "New")),
        3: df_map[gen]["rootDualBound"] - 1e-3 > df_map[gen]["dualBound"],
        4: (df_map[gen]["dualBound"] - 1e-3 > df_map[gen]["primalBound"]) & (df_map[gen]["dualBound"] / df_map[gen]["primalBound"] > 1 + 1e-3),
        5: df_map[gen]["primalBound"] > 1e20,
        6: 0 > df_map[gen]["vpcGenerationTime"],
        7: df_map[gen]["vpcGenerationTime"] - 1e-3 > df_map[gen]["rootDualBoundTime"],
        8: df_map[gen]["rootDualBoundTime"] - 1e-3 > df_map[gen]["terminationTime"],
        9: df_map[gen]["vpcGenerationTime"] - 1e-3 > df_map[gen]["bestSolutionTime"],
        10: df_map[gen]["bestSolutionTime"] - 1e-3 > df_map[gen]["terminationTime"]
    }
    for i, mask in masks.items():
        print(f"{gen} {i}: {mask.sum() / len(df_map[gen])}")

None 0: 0.0
None 1: 0.0
None 2: 0.0
None 3: 0.0
None 4: 0.0015752682878802796
None 5: 0.0
None 6: 0.0
None 7: 0.0
None 8: 0.0
None 9: 0.0
None 10: 0.0
New 0: 0.0
New 1: 0.0
New 2: 0.0
New 3: 0.0
New 4: 0.0018248175182481751
New 5: 0.0
New 6: 0.0
New 7: 0.0
New 8: 0.0
New 9: 0.0
New 10: 0.0
Farkas 0: 0.0
Farkas 1: 0.0
Farkas 2: 0.0
Farkas 3: 0.0
Farkas 4: 0.002344273275284662
Farkas 5: 0.0
Farkas 6: 0.0
Farkas 7: 0.0
Farkas 8: 0.0
Farkas 9: 0.0
Farkas 10: 0.0
All 0: 0.0
All 1: 0.0
All 2: 0.0
All 3: 0.0
All 4: 0.0028016109262826126
All 5: 0.0
All 6: 0.0
All 7: 0.0
All 8: 0.0
All 9: 0.0
All 10: 0.0


In [58]:
# it shouldn't be possible that dual bound > primal bound. this only happens when we use the saved primal bound, which was used to set the dual bound
df_map["Farkas"][masks[0]]

/var/folders/pb/p1sshdnx5sv12zwsxff8nrg40000gn/T/ipykernel_33109/3095696856.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_map["Farkas"][masks[0]]


,instanceIndex,seedIndex,vpcGenerator,terms,lpBound,disjunctiveDualBound,lpBoundPostVpc,rootDualBound,dualBound,primalBound,...,tighten_disjunction,tighten_matrix_perturbation,tighten_infeasible_to_feasible_term,tighten_feasible_to_infeasible_basis,instance,perturbation,degree,rows,cols,density


In [59]:
for gen in df_map:
    mask = (-1e20 > df_map[gen]["lpBound"]) | \
        (df_map[gen]["lpBound"] - 1e-3 > df_map[gen]["lpBoundPostVpc"]) | \
        ((df_map[gen]["lpBoundPostVpc"] - 1e-3 > df_map[gen]["disjunctiveDualBound"]) & (gen != "Farkas")) | \
        (df_map[gen]["rootDualBound"] - 1e-3 > df_map[gen]["dualBound"]) | \
        ((df_map[gen]["dualBound"] - 1e-3 > df_map[gen]["primalBound"]) & (df_map[gen]["dualBound"] / df_map[gen]["primalBound"] > 1 + 1e-3)) | \
        (df_map[gen]["primalBound"] > 1e20) | \
        (0 > df_map[gen]["vpcGenerationTime"]) | \
        (df_map[gen]["vpcGenerationTime"] - 1e-3 > df_map[gen]["rootDualBoundTime"]) | \
        (df_map[gen]["rootDualBoundTime"] - 1e-3 > df_map[gen]["terminationTime"]) | \
        (df_map[gen]["vpcGenerationTime"] - 1e-3 > df_map[gen]["bestSolutionTime"]) | \
        (df_map[gen]["bestSolutionTime"] - 1e-3 > df_map[gen]["terminationTime"])
    print(f"{gen}: {mask.sum() / len(df_map[gen])}")
    df_map[gen] = df_map[gen][~mask]

None: 0.0015752682878802796
New: 0.0018248175182481751
Farkas: 0.002344273275284662
All: 0.0028016109262826126


In [60]:
# merge the different data frames into one
join_cols = ["instance", "perturbation", "degree", "terms", "instanceIndex", "seedIndex"]
df = df_map[generators[0]].merge(df_map[generators[1]], on=join_cols, suffixes=(f" {generators[0]}", None))
for g1, g2 in zip(generators[1:-1], generators[2:]):
    df = df.merge(df_map[g2], on=join_cols, suffixes=(f" {g1}", None if g2 != generators[-1] else f" {g2}"))
df.head()

,instanceIndex,seedIndex,vpcGenerator None,terms,lpBound None,disjunctiveDualBound None,lpBoundPostVpc None,rootDualBound None,dualBound None,primalBound None,...,termRemainsFeasibleBasisInfeasible All,cutsChangedCoefficients All,feasibleTermsPrunedByBound All,tighten_disjunction All,tighten_matrix_perturbation All,tighten_infeasible_to_feasible_term All,tighten_feasible_to_infeasible_basis All,rows All,cols All,density All
0,0,0,None,64,-1.465365e+07,-1.465365e+07,-1.465365e+07,-1.462424e+07,-1.461219e+07,-1.461073e+07,...,0,0,0,0,0,0,0,812,1005,0.007121
1,1,0,None,64,-1.472593e+07,-1.472593e+07,-1.472593e+07,-1.469844e+07,-1.468368e+07,-1.468221e+07,...,32,0,0,1,1,1,1,812,1005,0.007121
2,2,0,None,64,-1.462379e+07,-1.462379e+07,-1.462379e+07,-1.460266e+07,-1.459395e+07,-1.459250e+07,...,20,0,0,1,1,1,1,812,1005,0.007121
3,3,0,None,64,-1.493177e+07,-1.493177e+07,-1.493177e+07,-1.490495e+07,-1.489484e+07,-1.489335e+07,...,23,0,0,1,1,1,1,812,1005,0.007121
4,0,0,None,64,3.157377e+03,3.157377e+03,3.157377e+03,3.467113e+03,3.663651e+03,3.664000e+03,...,0,0,0,0,0,0,0,285,504,0.007018


In [61]:
# get proportion of tests run to completion
len(generators) * len(df) / count_instances

0.504521328877531

In [62]:
def gap_closed(df, col):
    gap = abs(df[col] - df["lpBound None"]) / abs(df['primalBound None'] - df["lpBound None"])
    gap[(gap > 1) | (gap == np.nan)] = 1  # get corner cases
    return gap

# Function to map values based on a dictionary
def check_same_solution(row):
    # Create a tuple of the key based on the key_columns
    return same_solution[row["perturbation"], int(math.log2(row["degree"])), f'{row["instance"]}_{row["instanceIndex"]}']

In [63]:
# find the optimality gap closed by each generator
df["Disjunction (New)"] = gap_closed(df, "disjunctiveDualBound New")
df["Disjunction (Old)"] = gap_closed(df, "disjunctiveDualBound Farkas")
for g in generators:
    if g != "None":
        df[f"VPCs ({g})"] = gap_closed(df, f"lpBoundPostVpc {g}")        
    df[f"Root Cuts ({g})"] = gap_closed(df, f"rootDualBound {g}")

df["Root Optimality Gap Improvement"] = df["Root Cuts (Farkas)"] - df["Root Cuts (None)"] 
# df = df.dropna()

In [64]:
# find times without vpc generation
df["terminationTimeSansVpc None"] = df["terminationTime None"]
df["rootDualBoundTimeSansVpc None"] = df["rootDualBoundTime None"]
for gen in generators:
    if gen != "None":
        df[f"terminationTimeSansVpc {gen}"] = df[f"terminationTime {gen}"] - df[f"vpcGenerationTime {gen}"]
        df[f"rootDualBoundTimeSansVpc {gen}"] = df[f"rootDualBoundTime {gen}"] - df[f"vpcGenerationTime {gen}"]
    df[f"postRootTime {gen}"] = df[f"terminationTime {gen}"] - df[f"rootDualBoundTime {gen}"]
    if gen not in ["None", "New"]:
        df[f"terminationTimeImprovement {gen}"] = (df["terminationTime None"] - df[f"terminationTime {gen}"]) / df["terminationTime None"]
        df[f"terminationTimeSansVpcImprovement {gen}"] = (df["terminationTimeSansVpc None"] - df[f"terminationTimeSansVpc {gen}"]) / df["terminationTimeSansVpc None"]
        df[f"nodesImprovement {gen}"] = (df["nodes None"] - df[f"nodes {gen}"]) / df["nodes None"] 
        df[f"iterationsImprovement {gen}"] = (df["iterations None"] - df[f"iterations {gen}"]) / df["iterations None"] 
        df[f"terminationTimeRatio {gen}"] = df[f"terminationTime {gen}"] / df["terminationTime None"]
        df[f"terminationTimeSansVpcRatio {gen}"] = df[f"terminationTimeSansVpc {gen}"] / df["terminationTimeSansVpc None"]
        df[f"nodesRatio {gen}"] = df[f"nodes {gen}"] / df["nodes None"] 
        df[f"iterationsRatio {gen}"] = df[f"iterations {gen}"] / df["iterations None"]
        df[f"nodesImproves {gen}"] = df["nodes None"] > df[f"nodes {gen}"]
        df[f"terminationTimeImproves {gen}"] = df["terminationTime None"] > df[f"terminationTime {gen}"]
        df[f"terminationTimeSansVpcImproves {gen}"] = df["terminationTimeSansVpc None"] > df[f"terminationTimeSansVpc {gen}"]
        df[f"iterationsImproves {gen}"] = df["iterations None"] > df[f"iterations {gen}"]
        df[f'nodesWin{gen}'] = df['nodes None']*(1 - win_threshold) > df[f'nodes {gen}']
        df[f'terminationTimeWin{gen}'] = df['terminationTime None']*(1 - win_threshold) > df[f'terminationTime {gen}']
        df[f'terminationTimeSansVpcWin{gen}'] = df['terminationTimeSansVpc None']*(1 - win_threshold) > df[f'terminationTimeSansVpc {gen}']
        df[f'iterationsWin{gen}'] = df['iterations None']*(1 - win_threshold) > df[f'iterations {gen}']
        df[f'nodesWinNoneVs{gen}'] = df[f'nodes {gen}']*(1 - win_threshold) > df['nodes None']
        df[f'terminationTimeWinNoneVs{gen}'] = df[f'terminationTime {gen}']*(1 - win_threshold) > df['terminationTime None']
        df[f'terminationTimeSansVpcWinNoneVs{gen}'] = df[f'terminationTimeSansVpc {gen}']*(1 - win_threshold) > df['terminationTimeSansVpc None']
        df[f'iterationsWinNoneVs{gen}'] = df[f'iterations {gen}']*(1 - win_threshold) > df['iterations None']
df["bracket"] = ["short" if t <= short else "medium" if t <= medium else "long" for t in df["terminationTime None"]]
df["sameSolution"] = df.apply(check_same_solution, axis=1)

/var/folders/pb/p1sshdnx5sv12zwsxff8nrg40000gn/T/ipykernel_33109/3168479867.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'nodesWin{gen}'] = df['nodes None']*(1 - win_threshold) > df[f'nodes {gen}']
/var/folders/pb/p1sshdnx5sv12zwsxff8nrg40000gn/T/ipykernel_33109/3168479867.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'terminationTimeWin{gen}'] = df['terminationTime None']*(1 - win_threshold) > df[f'terminationTime {gen}']
/var/folders/pb/p1sshdnx5sv12zwsxff8nrg40000gn/T/ipykernel_33109/3168479867.py:24: P

In [65]:
# get sensitivity stats as ratios
for gen_name in generators:
    if gen_name == "None":
        continue
    df[f"infeasibleTermsRatio {gen_name}"] = df[f"infeasibleTerms {gen_name}"] / df[f"actualTerms {gen_name}"]
    df[f"infeasibleToFeasibleTermsRatio {gen_name}"] = df[f"infeasibleToFeasibleTerms {gen_name}"] / df[f"actualTerms {gen_name}"]
    df[f"zeroInfeasibleToFeasibleTerms {gen_name}"] = df[f"infeasibleToFeasibleTerms {gen_name}"] == 0
    df[f"feasibleToInfeasibleTermsRatio {gen_name}"] = df[f"feasibleToInfeasibleTerms {gen_name}"] / df[f"actualTerms {gen_name}"]

/var/folders/pb/p1sshdnx5sv12zwsxff8nrg40000gn/T/ipykernel_33109/393337708.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"infeasibleTermsRatio {gen_name}"] = df[f"infeasibleTerms {gen_name}"] / df[f"actualTerms {gen_name}"]
/var/folders/pb/p1sshdnx5sv12zwsxff8nrg40000gn/T/ipykernel_33109/393337708.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"infeasibleToFeasibleTermsRatio {gen_name}"] = df[f"infeasibleToFeasibleTerms {gen_name}"] / df[f"actualTerms {gen_name}"]
/var/folders/pb/p1sshdnx5sv12zwsxff8nrg40000gn

In [66]:
def optimality_gap(df, generator=None):
    if generator:
        return abs(df[f"primalBound {generator}"] - df[f"dualBound {generator}"]) / \
            abs(df[f"primalBound {generator}"])
    else:
        return abs(df[f"primalBound"] - df[f"dualBound"]) / abs(df[f"primalBound"])

In [67]:
# aleks filters
# df = df.loc[df["terms"] == df["actualTerms Farkas"]]
# df = df.loc[df["zeroInfeasibleToFeasibleTerms Farkas"]]

In [68]:
df.head()

,instanceIndex,seedIndex,vpcGenerator None,terms,lpBound None,disjunctiveDualBound None,lpBoundPostVpc None,rootDualBound None,dualBound None,primalBound None,...,zeroInfeasibleToFeasibleTerms New,feasibleToInfeasibleTermsRatio New,infeasibleTermsRatio Farkas,infeasibleToFeasibleTermsRatio Farkas,zeroInfeasibleToFeasibleTerms Farkas,feasibleToInfeasibleTermsRatio Farkas,infeasibleTermsRatio All,infeasibleToFeasibleTermsRatio All,zeroInfeasibleToFeasibleTerms All,feasibleToInfeasibleTermsRatio All
0,0,0,None,64,-1.465365e+07,-1.465365e+07,-1.465365e+07,-1.462424e+07,-1.461219e+07,-1.461073e+07,...,True,0.0,0.50000,0.000000,True,0.000000,0.50000,0.000000,True,0.000000
1,1,0,None,64,-1.472593e+07,-1.472593e+07,-1.472593e+07,-1.469844e+07,-1.468368e+07,-1.468221e+07,...,True,0.0,0.31250,0.187500,False,0.000000,0.31250,0.187500,False,0.000000
2,2,0,None,64,-1.462379e+07,-1.462379e+07,-1.462379e+07,-1.460266e+07,-1.459395e+07,-1.459250e+07,...,True,0.0,0.53125,0.156250,False,0.187500,0.53125,0.156250,False,0.187500
3,3,0,None,64,-1.493177e+07,-1.493177e+07,-1.493177e+07,-1.490495e+07,-1.489484e+07,-1.489335e+07,...,True,0.0,0.31250,0.328125,False,0.140625,0.31250,0.328125,False,0.140625
4,0,0,None,64,3.157377e+03,3.157377e+03,3.157377e+03,3.467113e+03,3.663651e+03,3.664000e+03,...,True,0.0,0.00000,0.000000,True,0.000000,0.00000,0.000000,True,0.000000


In [69]:
# set aside core columns and filter for all subsequent dataframes
group_cols = ["instance", "perturbation", "bracket", "degree", "terms"]
id_cols = ["instanceIndex"]

# keep the instance, perturbation, instanceIndex triples that exist for all combinations of degree and terms
# where VPC did not find the optimal solution
full_df = df.loc[df["Disjunction (New)"] < .9999]
triples = (full_df.groupby(
        ["instance", "perturbation", "instanceIndex"]
    ).size().reset_index().rename(columns={0: "count"}))
triples.head()

,instance,perturbation,instanceIndex,count
0,10teams,matrix,0,3
1,10teams,matrix,1,3
2,10teams,matrix,2,2
3,10teams,matrix,3,2
4,10teams,objective,0,4


In [70]:
# uncomment to filter for only the triples that exist for all combinations of degree and terms (and seed index)
# triples = triples[triples["count"] == len(degrees) * len(term_list) * len(seed_idxs)]
# full_df = full_df.merge(triples, on=["instance", "perturbation", "instanceIndex"])
full_df.to_csv(os.path.join(out_fldr, "cleaned_combined_complete.csv"), index=False, mode="w")

## Check Root Node Stats

In [71]:
def interleave(list_of_lists):
    return [item for sublist in zip(*list_of_lists) for item in sublist]

In [84]:
# additional filtering for dataframe on bounds
fields = ["Disjunction (New)", "Disjunction (Old)"] + [f"VPCs ({gen_name})" for gen_name in generators if gen_name != "None"] + \
    interleave([[f"Root Cuts ({gen_name})", f"terminationTime {gen_name}", f"nodes {gen_name}",
                 f"iterations {gen_name}", f"terminationTimeSansVpc {gen_name}", f"vpcGenerationTime {gen_name}", 
                 f"rootDualBoundTime {gen_name}"]
                for gen_name in generators]) + \
    interleave([[f"infeasibleTermsRatio {gen_name}", f"infeasibleToFeasibleTermsRatio {gen_name}",
                 f"zeroInfeasibleToFeasibleTerms {gen_name}", f"feasibleToInfeasibleTermsRatio {gen_name}"]
                for gen_name in generators if gen_name != "None"])

# now reduce bound_df to just the perturbed instances - make > -1 to include base instance
bound_df = full_df.loc[(full_df["instanceIndex"] > 0) & (full_df["Disjunction (Old)"] > .1), group_cols + id_cols + fields]

In [85]:
def geometric_mean(series, offset=1e-6):
    adjusted_series = series + offset  # Add a small offset to avoid zeros
    return np.exp(np.log(adjusted_series).mean())

# paper currently uses mean, but we can switch to geometric mean if we want
aggregations = {f: "mean" for f in fields}  # geometric_mean if f not in ["sameSolution"] else
aggregations["instance"] = "nunique"
aggregations["instanceIndex"] = "count"

In [86]:
# get gap closed by degree and term
out = bound_df.groupby(["degree", "terms"]).agg(aggregations).reset_index()
out.to_csv(os.path.join(out_fldr, "bound_table.csv"), index=False, mode="w")
out

,degree,terms,Disjunction (New),Disjunction (Old),VPCs (New),VPCs (Farkas),VPCs (All),Root Cuts (None),Root Cuts (New),Root Cuts (Farkas),...,infeasibleToFeasibleTermsRatio Farkas,infeasibleToFeasibleTermsRatio All,zeroInfeasibleToFeasibleTerms New,zeroInfeasibleToFeasibleTerms Farkas,zeroInfeasibleToFeasibleTerms All,feasibleToInfeasibleTermsRatio New,feasibleToInfeasibleTermsRatio Farkas,feasibleToInfeasibleTermsRatio All,instance,instanceIndex
0,1,4,0.284665,0.294949,0.136447,0.108094,0.107823,0.743479,0.749922,0.750418,...,0.001938,0.001938,1.0,0.992248,0.992248,0.0,0.012597,0.012597,28,129
1,1,64,0.277173,0.230163,0.154097,0.110926,0.114535,0.580678,0.610440,0.602555,...,0.006308,0.005729,1.0,0.927203,0.927203,0.0,0.023862,0.023862,55,261
2,4,4,0.295284,0.306864,0.121411,0.085346,0.085293,0.739165,0.741015,0.739363,...,0.000000,0.000000,1.0,1.000000,1.000000,0.0,0.027500,0.027500,29,100
3,4,64,0.295960,0.248630,0.150386,0.106305,0.109274,0.593950,0.616797,0.605236,...,0.007030,0.007030,1.0,0.935484,0.935484,0.0,0.020719,0.020719,49,186


In [87]:
# now break it down by type of perturbation
out = bound_df.groupby(["degree", "terms", "perturbation"]).agg(aggregations).reset_index()
out.to_csv(os.path.join(out_fldr, "bound_table_by_perturbation.csv"), index=False, mode="w")
out

,degree,terms,perturbation,Disjunction (New),Disjunction (Old),VPCs (New),VPCs (Farkas),VPCs (All),Root Cuts (None),Root Cuts (New),...,infeasibleToFeasibleTermsRatio Farkas,infeasibleToFeasibleTermsRatio All,zeroInfeasibleToFeasibleTerms New,zeroInfeasibleToFeasibleTerms Farkas,zeroInfeasibleToFeasibleTerms All,feasibleToInfeasibleTermsRatio New,feasibleToInfeasibleTermsRatio Farkas,feasibleToInfeasibleTermsRatio All,instance,instanceIndex
0,1,4,matrix,0.257777,0.290414,0.097940,0.084465,0.083635,0.802820,0.803275,...,0.007143,0.007143,1.0,0.971429,0.971429,0.0,0.010714,0.010714,15,35
1,1,4,objective,0.257862,0.308462,0.164017,0.143762,0.143662,0.702523,0.709714,...,0.000000,0.000000,1.0,1.000000,1.000000,0.0,0.000000,0.000000,24,60
2,1,4,rhs,0.359644,0.275771,0.127436,0.069475,0.069475,0.754668,0.765956,...,0.000000,0.000000,1.0,1.000000,1.000000,0.0,0.036765,0.036765,14,34
3,1,64,matrix,0.299254,0.224059,0.158741,0.086068,0.094013,0.604139,0.624142,...,0.015569,0.013747,1.0,0.855422,0.855422,0.0,0.043631,0.043631,41,83
4,1,64,objective,0.265477,0.237648,0.163247,0.129420,0.131194,0.523322,0.560427,...,0.000000,0.000000,1.0,1.000000,1.000000,0.0,0.000000,0.000000,40,98
5,1,64,rhs,0.268591,0.227326,0.138071,0.114061,0.115418,0.626598,0.657490,...,0.004428,0.004428,1.0,0.912500,0.912500,0.0,0.032581,0.032581,35,80
6,4,4,matrix,0.372201,0.352017,0.080175,0.044184,0.044184,0.834653,0.849869,...,0.000000,0.000000,1.0,1.000000,1.000000,0.0,0.054348,0.054348,12,23
7,4,4,objective,0.218144,0.266533,0.141558,0.113353,0.113279,0.666953,0.663522,...,0.000000,0.000000,1.0,1.000000,1.000000,0.0,0.000000,0.000000,25,61
8,4,4,rhs,0.478811,0.395719,0.103873,0.037741,0.037691,0.877209,0.879980,...,0.000000,0.000000,1.0,1.000000,1.000000,0.0,0.093750,0.093750,8,16
9,4,64,matrix,0.304017,0.211364,0.134648,0.053334,0.062541,0.609696,0.633004,...,0.013702,0.013702,1.0,0.820513,0.820513,0.0,0.027825,0.027825,23,39


## High Performing Root Stats

In [74]:
# example table for VPC strength
out.loc[out["degree"] != 16, ["degree", "terms", "perturbation"] + [c for c in out.columns if "VPCs" in c and "No" not in c]]

,degree,terms,perturbation,VPCs (New),VPCs (Farkas),VPCs (All),VPCs (Disjunction),VPCs (Matrix),VPCs (Term),VPCs (Basis)
0,1,4,matrix,0.003415,0.000085,0.000080,0.000086,0.000082,0.000085,0.000082
1,1,4,objective,0.003039,0.001810,0.001805,0.001808,0.001811,0.001803,0.001811
2,1,4,rhs,0.002059,0.000733,0.000733,0.000742,0.000738,0.000738,0.000738
3,1,64,matrix,0.004024,0.000030,0.000026,0.000030,0.000026,0.000030,0.000026
4,1,64,objective,0.003380,0.001144,0.001144,0.001141,0.001144,0.001144,0.001141
5,1,64,rhs,0.002052,0.000464,0.000464,0.000464,0.000464,0.000464,0.000464
6,4,4,matrix,0.003009,0.000021,0.000021,0.000021,0.000021,0.000021,0.000021
7,4,4,objective,0.003919,0.000266,0.000267,0.000267,0.000266,0.000267,0.000267
8,4,4,rhs,0.001184,0.000268,0.000268,0.000262,0.000261,0.000261,0.000262
9,4,64,matrix,0.008008,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006


In [75]:
# example table for root cut strength
out.loc[out["degree"] != 16, ["degree", "terms", "perturbation"] + [c for c in out.columns if "Root Cuts" in c and ("No" not in c or "None" in c)]]

,degree,terms,perturbation,Root Cuts (None),Root Cuts (New),Root Cuts (Farkas),Root Cuts (All),Root Cuts (Disjunction),Root Cuts (Matrix),Root Cuts (Term),Root Cuts (Basis)
0,1,4,matrix,0.385478,0.427697,0.433610,0.433870,0.452532,0.434577,0.432719,0.453482
1,1,4,objective,0.227477,0.290755,0.280216,0.276592,0.280784,0.281074,0.283286,0.282084
2,1,4,rhs,0.361085,0.387767,0.403542,0.366213,0.386416,0.388600,0.366975,0.390256
3,1,64,matrix,0.587282,0.594691,0.589064,0.598849,0.589879,0.584933,0.553784,0.551722
4,1,64,objective,0.283710,0.289211,0.289340,0.283048,0.294687,0.286118,0.286596,0.286339
5,1,64,rhs,0.367023,0.366815,0.352551,0.398597,0.352389,0.355860,0.371618,0.377384
6,4,4,matrix,0.282372,0.303090,0.281434,0.302163,0.281179,0.282219,0.301578,0.281271
7,4,4,objective,0.257414,0.319932,0.268482,0.269108,0.264374,0.267912,0.267027,0.271316
8,4,4,rhs,0.401654,0.400729,0.402744,0.404475,0.406715,0.403673,0.404910,0.404052
9,4,64,matrix,0.494018,0.658952,0.551633,0.493644,0.495453,0.492085,0.552211,0.552120


In [76]:
# example table for root cut generation time
out.loc[out["degree"] != 16, ["degree", "terms", "perturbation"] + [c for c in out.columns if "rootDualBoundTime" in c and ("No" not in c or "None" in c)]]

,degree,terms,perturbation,rootDualBoundTime None,rootDualBoundTime New,rootDualBoundTime Farkas,rootDualBoundTime All,rootDualBoundTime Disjunction,rootDualBoundTime Matrix,rootDualBoundTime Term,rootDualBoundTime Basis
0,1,4,matrix,0.804012,5.690215,0.907987,1.132910,0.861950,1.099766,0.856421,0.920438
1,1,4,objective,0.928106,3.960020,0.964559,0.999172,0.879016,0.878123,0.878775,0.880765
2,1,4,rhs,0.666270,3.490866,0.767218,0.811637,0.691941,0.685398,0.679782,0.716738
3,1,64,matrix,0.503209,14.763283,1.556552,2.413160,1.489902,2.254167,1.481378,2.032832
4,1,64,objective,0.616783,18.720847,1.329992,1.315494,1.236786,1.225892,1.227929,1.221089
5,1,64,rhs,0.533048,13.729910,1.137992,1.433787,1.052583,1.047485,1.059482,1.350479
6,4,4,matrix,0.553189,5.983054,0.750984,0.818002,0.660819,0.735258,0.663174,0.690750
7,4,4,objective,0.831665,4.192764,0.963185,0.969511,0.875583,0.892837,0.885076,0.894483
8,4,4,rhs,0.506615,2.566571,0.575095,0.522037,0.463354,0.470924,0.466380,0.467781
9,4,64,matrix,0.436028,22.715576,1.698011,2.144646,1.574241,2.038208,1.576972,1.976135


## Check Termination Stats

In [78]:
# additional filtering for dataframe on run time
fields = [f"terminationTime {gen}" for gen in generators] + \
         [f"terminationTimeImprovement {gen}" for gen in generators if gen not in ["None", "New"]]
# only check perturbed instances that solve to optimality and VPC didn't find optimal solution
mask = (df["Disjunction (New)"] < .9999) & (df["instanceIndex"] > 0) & (optimality_gap(df, "New") <= 1e-4) & \
    (optimality_gap(df, "None") <= 1e-4) & (optimality_gap(df, "Farkas") <= 1e-4) & \
       (df["terminationTime None"] > min_termination_time)

# create time dataframe
time_df = df.loc[mask, group_cols + id_cols + fields]

In [79]:
aggregations = {f"Average Time {gen}": (f"terminationTime {gen}", geometric_mean) for gen in generators} | \
    {f"Average Improvement {gen}": (f"terminationTimeImprovement {gen}", "mean") for gen in generators if gen not in ["None", "New"]} | \
    {"count": ("terminationTimeImprovement Farkas", "size")}

tmp = time_df.groupby(["instance", "perturbation", "degree", "terms"]).agg(**aggregations).reset_index()
tmp = tmp[(tmp["count"] > 1)]
tmp.to_csv(os.path.join(out_fldr, "high_perform_all.csv"), index=False, mode="w")
tmp.head()

,instance,perturbation,degree,terms,Average Time None,Average Time New,Average Time Farkas,Average Time All,Average Time Disjunction,Average Time Matrix,Average Time Term,Average Time Basis,Average Improvement Farkas,Average Improvement All,Average Improvement Disjunction,Average Improvement Matrix,Average Improvement Term,Average Improvement Basis,count
0,aflow30a,matrix,1,4,3.397194,6.908022,3.193351,2.461512,3.536989,3.335433,2.450183,2.442320,-0.014592,0.244497,-0.192570,-0.114798,0.221280,0.223497,9
1,aflow30a,matrix,1,64,5.262113,8.670610,5.423080,5.206116,4.875512,4.878356,4.999452,4.997785,-0.143554,-0.123315,-0.052529,-0.051905,-0.029195,-0.028607,8
2,aflow30a,rhs,1,4,2.071641,8.422406,2.870855,2.752171,2.240980,2.532260,2.320147,2.639981,-0.420380,-0.489788,-0.205667,-0.278145,-0.124080,-0.288904,3
3,aflow30a,rhs,1,64,3.124030,7.635770,3.190507,3.526027,3.366901,2.722832,2.450773,4.550764,-0.056778,-0.207908,-0.272866,0.085129,0.172113,-0.470320,3
4,aligninq,matrix,1,4,129.830683,130.857454,123.017751,113.325551,126.388712,126.584325,102.625044,126.977951,-0.113077,-0.018756,-0.076447,-0.078747,0.158329,-0.087549,10


In [80]:
def make_improvement_table(tmp, generator):
    
    # columns we always choose
    key_cols = ["degree", "terms", "perturbation", "instance"]
    time_cols = [f"Average Time {g}" for g in ["None", "New", "Farkas"]]
    
    # subset the ones we want
    all_df = tmp[
        key_cols + time_cols + [f"Average Time {generator}", f"Average Improvement Farkas", f"Average Improvement {generator}", "count"]
    ].sort_values(f"Average Improvement {generator}", ascending=False)
    all_df = all_df[all_df[f"Average Improvement {generator}"] > 0]
    best_df = all_df.loc[
        all_df.groupby(['perturbation', 'degree', 'terms'])[f'Average Improvement {generator}'].idxmax()
    ].sort_values(f"Average Improvement {generator}", ascending=False)
    
    # save all the winners
    all_df.to_csv(os.path.join(out_fldr, f"high_perform_{generator.lower()}.csv"), index=False, mode="w")
    
    # return just the best
    return all_df, best_df

In [81]:
all_df, best_df = make_improvement_table(tmp, "Disjunction")
best_df.loc[[285, 392, 213, 373, 186, 172], [c for c in best_df.columns if "Improvement" not in c]]

,degree,terms,perturbation,instance,Average Time None,Average Time New,Average Time Farkas,Average Time Disjunction,count
285,1,4,matrix,neos-860300,129.235105,97.853468,70.466980,40.607542,10
392,1,4,objective,ran13x13,35.537751,14.976852,17.016891,13.095101,10
213,4,4,rhs,neos-3046615-murg,231.783260,168.801212,200.152352,150.464393,2
373,4,64,rhs,pg5_34,3.150931,71.660436,2.063292,1.944108,8
186,16,4,matrix,n7-3,8.888611,30.491732,13.156323,5.293075,2
172,16,64,objective,misc07,83.671824,47.174409,41.371435,28.676896,10


In [82]:
all_df, best_df = make_improvement_table(tmp, "Matrix")
best_df.loc[best_df["perturbation"] == "matrix", [c for c in best_df.columns if "Improvement" not in c]].sort_values(["degree", "terms"])

,degree,terms,perturbation,instance,Average Time None,Average Time New,Average Time Farkas,Average Time Matrix,count
285,1,4,matrix,neos-860300,129.235105,97.853468,70.466980,50.045030,10
388,1,64,matrix,ran13x13,22.612178,47.155872,20.394313,15.924265,10
332,4,4,matrix,neos18,5.268503,11.776036,4.612012,3.310361,9
75,4,64,matrix,g200x740,26.799084,62.360644,33.217829,20.611519,2
186,16,4,matrix,n7-3,8.888611,30.491732,13.156323,5.340057,2
296,16,64,matrix,neos-911970,29.294123,26.230940,26.757343,19.561260,6


In [83]:
all_df, best_df = make_improvement_table(tmp, "Term")
best_df = best_df.loc[(best_df["perturbation"] == "matrix") | (best_df["perturbation"] == "rhs"),
                      [c for c in best_df.columns if "Improvement" not in c]]
best_df.loc[[285, 373, 410, 296, 131, 341]].sort_values(["degree", "terms"])

,degree,terms,perturbation,instance,Average Time None,Average Time New,Average Time Farkas,Average Time Term,count
285,1,4,matrix,neos-860300,129.235105,97.853468,70.466980,39.316548,10
341,1,64,rhs,neos18,13.655170,638.616928,16.640351,9.637685,2
410,4,4,matrix,rout,10.085790,8.247897,10.493955,6.188331,10
373,4,64,rhs,pg5_34,3.150931,71.660436,2.063292,1.975149,8
131,16,4,rhs,mas76,47.049209,63.572653,44.410446,38.407041,4
296,16,64,matrix,neos-911970,29.294123,26.230940,26.757343,19.431839,6


In [84]:
all_df, best_df = make_improvement_table(tmp, "Basis")
best_df = best_df.loc[(best_df["perturbation"] == "matrix") | (best_df["perturbation"] == "rhs"),
                      [c for c in best_df.columns if "Improvement" not in c]]
best_df.loc[[285, 341, 195, 390, 296, 295]].sort_values(["degree", "terms"])

,degree,terms,perturbation,instance,Average Time None,Average Time New,Average Time Farkas,Average Time Basis,count
285,1,4,matrix,neos-860300,129.235105,97.853468,70.466980,37.981369,10
341,1,64,rhs,neos18,13.655170,638.616928,16.640351,9.832116,2
195,4,4,rhs,neos-1445743,43.887964,2706.963349,36.087721,24.296440,4
390,4,64,matrix,ran13x13,54.497417,66.367305,77.418971,42.131783,10
295,16,4,matrix,neos-911970,19.315416,18.745702,17.694233,10.563624,5
296,16,64,matrix,neos-911970,29.294123,26.230940,26.757343,16.078533,6


In [87]:
all_df, best_df = make_improvement_table(tmp, "All")
# swap 335 and 120 for poster vs paper
best_df.loc[[285, 335, 35, 390, 301, 172], [c for c in best_df.columns if "Improvement" not in c]].sort_values(["degree", "terms"])

,degree,terms,perturbation,instance,Average Time None,Average Time New,Average Time Farkas,Average Time All,count
285,1,4,matrix,neos-860300,129.235105,97.853468,70.466980,56.302859,10
335,1,64,objective,neos18,26.496431,51.959339,24.825103,20.318973,3
35,4,4,rhs,blp-ir98,12.691349,143.314590,8.665618,7.336168,2
390,4,64,matrix,ran13x13,54.497417,66.367305,77.418971,45.969478,10
301,16,4,objective,neos-911970,102.516404,69.198447,139.244088,41.855894,2
172,16,64,objective,misc07,83.671824,47.174409,41.371435,35.810067,10


In [86]:
best_df

,degree,terms,perturbation,instance,Average Time None,Average Time New,Average Time Farkas,Average Time All,Average Improvement Farkas,Average Improvement All,count
172,16,64,objective,misc07,83.671824,47.174409,41.371435,35.810067,0.488652,0.555762,10
285,1,4,matrix,neos-860300,129.235105,97.853468,70.466980,56.302859,0.324082,0.507047,10
301,16,4,objective,neos-911970,102.516404,69.198447,139.244088,41.855894,-1.864602,0.483036,2
188,4,4,objective,n7-3,3.373049,6.932989,1.356729,1.353820,0.420758,0.421294,3
35,4,4,rhs,blp-ir98,12.691349,143.314590,8.665618,7.336168,0.306335,0.398933,2
346,1,4,objective,pg,26.701193,20.936635,15.808465,17.704975,0.394580,0.322339,10
120,1,64,matrix,mas76,17.747258,38.971226,13.788161,9.943748,0.021632,0.298938,4
349,4,64,objective,pg,10.368741,46.866594,7.412535,7.476839,0.277314,0.266978,10
74,4,4,matrix,g200x740,41.138180,57.774963,36.850937,29.597412,0.104088,0.262586,2
290,1,4,rhs,neos-860300,10.123230,78.902810,16.034225,7.102350,-0.727623,0.252821,7


In [93]:
# find ratios of all vs farkas
# ijoc santanu and prachi paper on adding one cut and tree blows up
# are the cuts getting stronger?
# does time (excluding cut generation) improve when tightening improves